# 🚀 Welcome to Your ADK Adventure - MultiAgents! 🚀

Welcome back, Agent Architect! This notebook dives into the heart of the Google Agent Development Kit (ADK): orchestrating teams of specialized agents to tackle complex, multi-step problems that a single agent cannot handle alone.

By the end of this session, you will be an expert in advanced agentic workflows:

- **SequentialAgent**: You'll learn to chain agents together, creating pipelines where the output of one agent becomes the input for the next.

- **LoopAgent**: You'll build iterative systems where agents can plan, critique, and refine their work until a specific goal is met, making them "perfectionists."

- **ParallelAgent**: You'll unleash efficiency by running multiple agents simultaneously and then synthesizing their collective findings into a single, comprehensive answer.

- **The Router**: You will construct a "master" router agent that intelligently analyzes a user's request and delegates it to the correct agent or workflow.

Let's get this adventure started!

## Author

HI, I'm Qingyue (Annie) Wang, a developer advocate and AI engineer at **Google**, passionate about helping developers build with AI and cloud technologies :)


If you have questions with this notebook, contact me on [LinkedIn](https://www.linkedin.com/in/anniewangtech/) , [X](https://twitter.com/anniewangtech) or email anniewangtech0510@Gmail.com


```

  (\__/)
  (•ㅅ•)
  /づ  📚       Enjoy learning AI Agents :)


```


-------------
### 🎁 🛑 Important Prerequisite: Setup Your Environment! 🛑 🎁
-----------------------------------------------------------------------------

👉 **Get Your API Key HERE**: https://codelabs.developers.google.com/onramp/instructions#0

 -----------------------------------------------------------------------------

```
 ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️  ⬆️
   /\_/\     /\_/\     /\_/\      /\_/\       /\_/\
  ( ^_^ )   ( -.- )   ( >_< )   ( =^.^= )    ( o_o )             
```


## Part 0: Setup & Authentication 🔑

First things first, let's get all our tools ready. This step installs the necessary libraries and securely configures your Google API key so your agents can access the power of Gemini.

In [1]:
# !pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent, SequentialAgent, LoopAgent, ParallelAgent
from google.adk.tools import google_search, ToolContext
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [13]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

Enter your Google API Key: ··········
✅ API Key configured successfully! Let the fun begin.


In [3]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"

---
## Part 1: Multi-Agent Mayhem - Sequential Workflows 🧠→🤖→🤖

You've mastered single agents and memory. Now for the most advanced topic: making agents **work together in a sequence**.

Some tasks are too complex for one agent. A user might ask, "Find me a great restaurant and then tell me how to get there." This requires two different skills: food recommendation and navigation.

We'll build a system that can handle this by:
1.  Creating a new `transportation_agent` 🚗.
2.  Teaching our `router_agent` 🧠 to recognize these special "combo" requests.
3.  Writing Python code (the "orchestrator") that runs the agents in a sequence, passing the output of the first agent to the second.

```
                    +---------------------+
                    |    User Query 🗣️     |
                    +----------+----------+
                               |
                               v
                    +---------------------+
                    |   Router Agent 🤖    |
                    |  (Classify Request) |
                    +----------+----------+
                               |
          +--------------------+----------------------+
          |                    |                      |
          v                    v                      v
  +----------------+   +--------------------+  +----------------------+
  |  foodie_agent  |   | weekend_guide_agent|  |  day_trip_agent      |
  |  🍣 Food Search |   | 🎉 Event Discovery |  | 🧳 Trip Planner       |
  +----------------+   +--------------------+  +----------------------+
          |
          v
  +----------------------------+            (if combo request)
  |  Restaurant Recommendation |---------------------------+
  |  ex: "Best sushi is at X"  |                           |
  +----------------------------+                           v
                                                        +-----------------------+
                                                        | transportation_agent  |
                                                        | 🚗 Get Directions      |
                                                        +-----------------------+
                                                        | Input: origin, place  |
                                                        | Output: directions    |
                                                        +-----------------------+

Final Output: 🍽️ Recommendation + 🚗 Route Info
```


In [9]:
# --- Agent Definitions for our Specialist Team ---
# --- Agent Definition ---

day_trip_agent = Agent(
    name="day_trip_agent",
    model="gemini-2.5-flash",
    description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
    instruction="""
    You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

    Your Mission:
    Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

    Guidelines:
    1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
    2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
    3. **Real-Time Focus**: Search for current operating hours and special events.
    4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

    RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
    """,
    tools=[google_search]
)

foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are an expert food critic. Your goal is to find the absolute best food, restaurants, or culinary experiences based on a user's request. When you recommend a place, state its name clearly. For example: 'The best sushi is at **Jin Sho**.'"
)

weekend_guide_agent = Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

transportation_agent = Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a navigation assistant. Given a starting point and a destination, provide clear directions on how to get from the start to the end."
)

# --- The Brain of the Operation: The Router Agent ---
# We update the router's instructions to know about the new 'combo' task.
router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_combo': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We'll create a dictionary of all our individual worker agents
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "transportation_agent": transportation_agent, # Add the new agent!
}

print("🤖 Agent team assembled for sequential workflows!")

🤖 Agent team assembled for sequential workflows!


In [10]:
# --- Let's Test the Sequential Workflow! ---

async def run_sequential_app():
    queries = [
        "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the COMBO
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        if chosen_route == 'find_and_navigate_combo':
            print("\n--- Starting Find and Navigate Combo Workflow ---")

            # STEP 2a: Run the foodie_agent first
            foodie_session = await session_service.create_session(app_name=foodie_agent.name, user_id=my_user_id)
            foodie_response = await run_agent_query(foodie_agent, query, foodie_session, my_user_id)

            # STEP 2b: Extract the destination from the first agent's response
            # (This is a simple regex, a more robust solution might use a structured output format)
            match = re.search(r'\*\*(.*?)\*\*', foodie_response)
            if not match:
                print("🚨 Could not determine the restaurant name from the response.")
                continue
            destination = match.group(1)
            print(f"💡 Extracted Destination: {destination}")

            # STEP 2c: Create a new query and run the transportation_agent
            directions_query = f"Give me directions to {destination} from the Palo Alto Caltrain station."
            print(f"\n🗣️ New Query for Transport Agent: '{directions_query}'")
            transport_session = await session_service.create_session(app_name=transportation_agent.name, user_id=my_user_id)
            await run_agent_query(transportation_agent, directions_query, transport_session, my_user_id)

            print("--- Combo Workflow Complete ---")

        elif chosen_route in worker_agents:
            # This is a simple, single-agent route
            worker_agent = worker_agents[chosen_route]
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_sequential_app()


🗣️ Processing New Query: 'I want to eat the best sushi in Palo Alto.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'cf9ff91e-9da1-4809-9dd7-b8d28209a7db'...
🚦 Router has selected route: 'foodie_agent'

🚀 Running query for agent: 'foodie_agent' in session: '10d1cabd-52f1-40b7-8132-7398c2dcf690'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""For the absolute best sushi experience in Palo Alto, I recommend **Iki Omakase**.

Iki Omakase offers a modern, refined sushi omakase concept led by Chef Jiabo Li, whose culinary philosophy is deeply rooted in his journey as a dedicated sushi shokunin. The restaurant emphasizes innate artistry, culinary mastery, tireless pursuit of excellence, and sincere hospitality, presenting creations using the finest globally and locally sourced ingredients. It reflects a unique Bay Area style with a firm foundation in Edomae sushi, making it an exceptional 

For the absolute best sushi experience in Palo Alto, I recommend **Iki Omakase**.

Iki Omakase offers a modern, refined sushi omakase concept led by Chef Jiabo Li, whose culinary philosophy is deeply rooted in his journey as a dedicated sushi shokunin. The restaurant emphasizes innate artistry, culinary mastery, tireless pursuit of excellence, and sincere hospitality, presenting creations using the finest globally and locally sourced ingredients. It reflects a unique Bay Area style with a firm foundation in Edomae sushi, making it an exceptional choice for those seeking the highest quality and a dedicated sushi experience.

Another highly regarded option for excellent sushi in Palo Alto is **Jin Sho**. Known as a favorite of Steve Jobs, Jin Sho is praised for its fresh fish, traditional sushi offerings, and authentic atmosphere. It's a "down-to-earth regular Japanese restaurant" offering great sushi, sashimi, and other Japanese delicacies.

--------------------------------------------------


🗣️ Processing New Query: 'Are there any cool outdoor concerts this weekend?'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'b4d8f0eb-9551-4012-bbff-e7afa556e47e'...
🚦 Router has selected route: 'weekend_guide_agent'

🚀 Running query for agent: 'weekend_guide_agent' in session: '5dbcc248-955a-4b6c-a1f6-157a6cac26a5'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""This upcoming weekend, January 10th and 11th, 2026, offers a variety of outdoor musical experiences across different parts of the world. While outdoor concerts might be less common in some Northern Hemisphere locations due to winter, there are exciting festivals and shows in warmer climates and even some unique winter outdoor events.

Here are a few "cool" outdoor concerts and festivals happening this weekend:

**International Highlights:**

*   **Milk + Cookies Johannesburg

This upcoming weekend, January 10th and 11th, 2026, offers a variety of outdoor musical experiences across different parts of the world. While outdoor concerts might be less common in some Northern Hemisphere locations due to winter, there are exciting festivals and shows in warmer climates and even some unique winter outdoor events.

Here are a few "cool" outdoor concerts and festivals happening this weekend:

**International Highlights:**

*   **Milk + Cookies Johannesburg** in Johannesburg, South Africa, on Saturday, January 10th. This one-day festival features artists such as Gunna, Majid Jordan, and Elmiene.
*   **Closer to the Sun** in Puerto Aventuras, Mexico, running from January 10th to 13th. The lineup includes Rebelution, Dirty Heads, and Slightly Stoopid.
*   **Day Zero Tulum** in Playa del Carmen, Mexico, on Saturday, January 10th, promises a transformative experience in the mystical jungles of Tulum with artists like Acid Pauli, Cinthie, and Damian Lazarus.
*   **Ocean Sounds Festival** in Newhaven, Victoria, Australia, on Saturday, January 10th, features The Teskey Brothers, The Presets, and Thelma Plum.
*   **Insônia Festival** in Rio de Janeiro, Brazil, takes place on January 10th and 11th, with performances by Shanti People, Major7, and Becker.
*   The **Kirstenbosch Summer Sunset Concerts** in Cape Town, South Africa, are an ongoing series. While not part of the main series, Calum Scott will be performing outdoors on January 14, 2026.

**United States Highlights:**

*   **FYA Fest** in Orlando, Florida, will be held on January 10th and 11th, featuring bands like Hatebreed, Drain, and Gorilla Biscuits.
*   The **Everglades Roots Festival 2026** in Ochopee, Florida, is scheduled from January 9th to 11th.
*   In Winter Park, Colorado, there's a **FREE concert with Big Gigantic** at the Rendezvous Event Center on Saturday, January 10th, from 5:00 PM to 8:00 PM, followed by fireworks. Lizzy Jane is also scheduled to perform. Please note that this event is currently sold out, but it's advisable to check for any additional ticket releases.
*   The **South Beach Jazz Festival** in Miami Beach, Florida, runs from January 8th to 11th. Specifically, the Spanish Harlem Orchestra with special guest Etienne Charles will perform on Saturday, January 10th, from 8:00 PM to 11:00 PM at the Miami Beach Bandshell. On Sunday, January 11th, the Lemon City Trio will perform at 1111 Lincoln Road from 12:45 PM to 1:45 PM.
*   **Garden AMP** in Garden Grove, California, is hosting a tribute event on Saturday, January 10th, 2026, featuring tributes to My Chemical Romance, Paramore, and Misfits, along with Finding Emo - Taste The Khaos.
*   For those in the Northeast, The Stone Pony in Asbury Park, New Jersey, is hosting **Slippery When Wet NJ (Bon Jovi Tribute)** on Friday, January 9th. While The Stone Pony is known for its outdoor Summer Stage, the specific calendar entry for this event does not explicitly state it is outdoors.

Since "cool" can be subjective and location is a key factor for outdoor events, please let me know your preferred location, and I can try to find more tailored recommendations for you!

--------------------------------------------------


🗣️ Processing New Query: 'Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '3d82d0e5-66f5-4d3c-b04b-e011f89695e9'...
🚦 Router has selected route: 'find_and_navigate_combo'

--- Starting Find and Navigate Combo Workflow ---

🚀 Running query for agent: 'foodie_agent' in session: 'e6450783-c3ea-424a-ab5f-366bebd7ee19'...
EVENT: model_version='gemini-2.5-flash' content=Content(
  parts=[
    Part(
      text="""As an expert food critic, when it comes to the absolute best sushi in Palo Alto for a truly refined and modern experience, I highly recommend **Iki Omakase**. This establishment is lauded for its dedication to the art of sushi by Chef Jiabo Li, offering an unparalleled omakase journey with globally and locally sourced ingredients.

Unfortunately, despite multiple attempts, I was un

As an expert food critic, when it comes to the absolute best sushi in Palo Alto for a truly refined and modern experience, I highly recommend **Iki Omakase**. This establishment is lauded for its dedication to the art of sushi by Chef Jiabo Li, offering an unparalleled omakase journey with globally and locally sourced ingredients.

Unfortunately, despite multiple attempts, I was unable to pinpoint the precise street address for Iki Omakase in Palo Alto from the available information. While it is consistently referred to as being in "Palo Alto" and offering "modern sushi omakase", a specific numerical street address was not explicitly listed in my search results.

Therefore, while I can confidently recommend **Iki Omakase** for an exceptional sushi experience, you will need to verify their exact address directly (perhaps through their reservation platform or by contacting them) to get the most accurate directions from the Caltrain station.

--------------------------------------------------

💡 Extracted Destination: Iki Omakase

🗣️ New Query for Transport Agent: 'Give me directions to Iki Omakase from the Palo Alto Caltrain station.'

🚀 Running query for agent: 'transportation_agent' in session: '9f87d860-d6da-4d3d-b1d5-00927411bbfe'...

--------------------------------------------------
✅ Final Response:


An error occurred: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash\nPlease retry in 35.793516757s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.5-flash'}, 'quotaValue': '5'}]}, {'@type': 'type.googleapis.com/google.rpc.RetryInfo', 'retryDelay': '35s'}]}}

--------------------------------------------------

--- Combo Workflow Complete ---


---
### Part 2 (The ADK Way): Multi-Agent Mayhem with `SequentialAgent` 🧠→⛓️→🤖

You've seen how to manually link agents together with custom Python code. It works, but it can get complicated. Now, let's refactor our workflow to use a powerful, built-in ADK feature designed specifically for this: the **`SequentialAgent`**.

The `SequentialAgent` is a *workflow agent*. It's not powered by an LLM itself; instead, its only job is to execute a list of other agents in a strict, predefined order.

The real magic ✨ is how it passes information. The ADK uses a shared `state` dictionary that each agent in the sequence can read from and write to.

**Our New Workflow:**

1.  **Foodie Agent**: Finds the restaurant and saves the name to `state['destination']`.
2.  **Transportation Agent**: Automatically reads `state['destination']` and uses it to find directions.

This means we no longer need custom Python code to extract text or build new queries! The ADK handles the plumbing for us.

```
+-------------------------------+
|  find_and_navigate_agent 🧭   |
| SequentialAgent:             |
| 1. Find destination          |
| 2. Get directions            |
+---------------+---------------+
                |
     +----------+----------+
     |                     |
     v                     v
+----------------+   +------------------------+
| foodie_agent 🍣 |   | transportation_agent 🚗 |
| Finds place     |   | Uses {destination}     |
| Output: 'Jin Sho'|   | Output: Directions     |
+----------------+   +------------------------+

Final Output: 🍣 Restaurant + 🚗 Route
```

In [14]:
# --- Agent Definitions for our Specialist Team (Refactored for Sequential Workflow) ---

# ✨ CHANGE 1: We tell foodie_agent to save its output to the shared state.
# Note the new `output_key` and the more specific instruction.
foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment and nothing else.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="destination"  # ADK will save the agent's final response to state['destination']
)

# ✨ CHANGE 2: We tell transportation_agent to read from the shared state.
# The `{destination}` placeholder is automatically filled by the ADK from the state.
transportation_agent = Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are a navigation assistant. Given a destination, provide clear directions.
    The user wants to go to: {destination}.

    Analyze the user's full original query to find their starting point.
    Then, provide clear directions from that starting point to {destination}.
    """,
)

# ✨ CHANGE 3: Define the SequentialAgent to manage the workflow.
# This agent will run foodie_agent, then transportation_agent, in that exact order.
find_and_navigate_agent = SequentialAgent(
    name="find_and_navigate_agent",
    sub_agents=[foodie_agent, transportation_agent],
    description="A workflow that first finds a location and then provides directions to it."
)

weekend_guide_agent = Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

# --- The Brain of the Operation: The Router Agent ---

# We update the router to know about our new, powerful SequentialAgent.
router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_agent': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We create a dictionary of all our executable agents for easy lookup.
# This now includes our powerful new workflow agent!
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "find_and_navigate_agent": find_and_navigate_agent, # Add the new sequential agent
}

print("🤖 Agent team assembled with a SequentialAgent workflow!")

🤖 Agent team assembled with a SequentialAgent workflow!


In [15]:
# --- Let's Test the Streamlined Workflow! ---

async def run_sequential_app():
    queries = [
        "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the SequentialAgent
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        # This logic is now much simpler! The SequentialAgent is treated just like any other worker.
        if chosen_route in worker_agents:
            worker_agent = worker_agents[chosen_route]
            print(f"--- Handing off to {worker_agent.name} ---")
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
            print(f"--- {worker_agent.name} Complete ---")
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_sequential_app()


🗣️ Processing New Query: 'I want to eat the best sushi in Palo Alto.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'ffe379ea-3ebb-4c90-957b-56946d8f16cb'...
🚦 Router has selected route: 'An error occurred: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {error: {code: 429, message: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 30.502716344s., status: RESOURCE_EXHAUSTED, details: [{@type: type.googleapis.com/google.rpc.Help, links: [{description: Learn more about Gemin

### Running the Streamlined App

Notice how much simpler the code below is. There is no longer a special `if chosen_route == 'find_and_navigate_combo':` block with custom logic.

The `find_and_navigate_agent` is now a self-contained unit. We can treat it just like any other agent, hand it the query, and trust the `SequentialAgent` to handle all the internal steps. This makes our main application code much cleaner and easier to read.

---
## Iterative Ideas with `LoopAgent` 🧠→🔁→🤖

Sometimes a task isn't a straight line; it's a loop of refinement. A user might ask for a plan, but have constraints that require checking and re-planning. For this, the ADK provides the **`LoopAgent`**.

The `LoopAgent` executes a sequence of sub-agents repeatedly until a condition is met. This is perfect for workflows involving trial and error, like planning a trip with a tight schedule.

**Our New Workflow: The Perfectionist Planner**

1. **Planner Agent**: Proposes an itinerary (e.g., a museum and a restaurant).
2. **Critic Agent**: Checks the plan against a constraint (e.g., "Is the travel time between these two places less than 30 minutes?").
3. **Refiner Agent**: If the critic finds a problem, this agent takes the feedback and creates a new, improved plan. If the critic is happy, it calls a special `exit_loop` tool to stop the process.

The `LoopAgent` manages this cycle, ensuring we don't get stuck in an infinite loop by setting a `max_iterations` limit.

```
+-------------------------------+
|  iterative_planner_agent 🔁   |
| SequentialAgent:              |
| 1. Propose Plan               |
| 2. Refine in loop (≤ 3 times) |
+---------------+---------------+
                |
     +----------+----------+
     |                     |
     v                     v
+----------------+   +-----------------------+
| planner_agent  |   | refinement_loop 🔁     |
| Propose plan   |   | LoopAgent             |
| e.g., Activity +  | 1. Critic (time check) |
| Restaurant       | 2. Refiner (fix/exit)   |
+----------------+   +-----------------------+

Uses shared state: {current_plan}, {criticism}
Exits when: "Plan is feasible..."

```

In [16]:
# --- Agent Definitions for an Iterative Workflow ---

# A tool to signal that the loop should terminate
COMPLETION_PHRASE = "The plan is feasible and meets all constraints."
def exit_loop(tool_context: ToolContext):
  """Call this function ONLY when the plan is approved, signaling the loop should end."""
  print(f"  [Tool Call] exit_loop triggered by {tool_context.agent_name}")
  tool_context.actions.escalate = True
  return {}

# Agent 1: Proposes an initial plan
planner_agent = Agent(
    name="planner_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are a trip planner. Based on the user's request, propose a single activity and a single restaurant. Output only the names, like: 'Activity: Exploratorium, Restaurant: La Mar'.",
    output_key="current_plan"
)

# Agent 2 (in loop): Critiques the plan
critic_agent = Agent(
    name="critic_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction=f"""You are a logistics expert. Your job is to critique a travel plan. The user has a strict constraint: total travel time must be short.
    Current Plan: {{current_plan}}
    Use your tools to check the travel time between the two locations.
    IF the travel time is over 45 minutes, provide a critique, like: 'This plan is inefficient. Find a restaurant closer to the activity.'
    ELSE, respond with the exact phrase: '{COMPLETION_PHRASE}'""",
    output_key="criticism"
)

# Agent 3 (in loop): Refines the plan or exits
refiner_agent = Agent(
    name="refiner_agent", model="gemini-2.5-flash", tools=[google_search, exit_loop],
    instruction=f"""You are a trip planner, refining a plan based on criticism.
    Original Request: {{session.query}}
    Critique: {{criticism}}
    IF the critique is '{COMPLETION_PHRASE}', you MUST call the 'exit_loop' tool.
    ELSE, generate a NEW plan that addresses the critique. Output only the new plan names, like: 'Activity: de Young Museum, Restaurant: Nopa'.""",
    output_key="current_plan"
)

# ✨ The LoopAgent orchestrates the critique-refine cycle ✨
refinement_loop = LoopAgent(
    name="refinement_loop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=3
)

# ✨ The SequentialAgent puts it all together ✨
iterative_planner_agent = SequentialAgent(
    name="iterative_planner_agent",
    sub_agents=[planner_agent, refinement_loop],
    description="A workflow that iteratively plans and refines a trip to meet constraints."
)

print("🤖 Agent team updated with an iterative LoopAgent workflow!")

🤖 Agent team updated with an iterative LoopAgent workflow!


---
## Parallel Power with `ParallelAgent` 🧠→⚡️→🤖🤖🤖

What if a user wants to find multiple, unrelated things at once? "Find me a museum, a concert, AND a restaurant." Running these searches one by one is slow and inefficient.

Enter the **`ParallelAgent`**. This workflow agent executes a list of sub-agents *concurrently*, dramatically speeding up tasks that can be performed independently.

**Our New Workflow: The Multi-Researcher**

1.  **Parallel Agent**: Simultaneously runs three specialist agents:
    - `MuseumFinderAgent`: Finds a museum.
    - `ConcertFinderAgent`: Finds a concert.
    - `FoodieAgent`: Finds a restaurant.
2.  **Synthesis Agent**: Once all three parallel searches are complete, this final agent gathers the results (which were saved to the shared `state`) and formats them into a single, neat summary for the user.

This pattern lets us get a lot of work done, fast! 🚀

```
+-------------------------------+
|  parallel_planner_agent ⚡     |
| SequentialAgent:              |
| 1. Run parallel research      |
| 2. Synthesize results         |
+---------------+---------------+
                |
     +----------+----------------------+
     |                                 |
     v                                 v
+-------------------------+       +-----------------------------+
| parallel_research_agent ⚡   |   | synthesis_agent 📋          |
| ParallelAgent:              |   | Combine results            |
| - museum_finder_agent 🖼️     |   | Output: Bulleted summary   |
| - concert_finder_agent 🎵    |   +-----------------------------+
| - restaurant_finder_agent 🍽️ |
+-------------------------+

Final Output:
• Museum: XYZ  
• Concert: Artist at Venue  
• Restaurant: ABC
```

In [17]:
# --- Agent Definitions for a Parallel Workflow ---

# Specialist Agent 1
museum_finder_agent = Agent(
    name="museum_finder_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are a museum expert. Find the best museum based on the user's query. Output only the museum's name.",
    output_key="museum_result"
)

# Specialist Agent 2
concert_finder_agent = Agent(
    name="concert_finder_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are an events guide. Find a concert based on the user's query. Output only the concert name and artist.",
    output_key="concert_result"
)

# We can reuse our foodie_agent for the third parallel task!
# Just need to give it a new output_key for this workflow.
# restaurant_finder_agent = foodie_agent.copy(update={"output_key": "restaurant_result"})
restaurant_finder_agent = Agent(
    name="restaurant_finder_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="restaurant_result" # Set the correct output key for this workflow
)


# ✨ The ParallelAgent runs all three specialists at once ✨
parallel_research_agent = ParallelAgent(
    name="parallel_research_agent",
    sub_agents=[museum_finder_agent, concert_finder_agent, restaurant_finder_agent]
)

# Agent to synthesize the parallel results
synthesis_agent = Agent(
    name="synthesis_agent", model="gemini-2.5-flash",
    instruction="""You are a helpful assistant. Combine the following research results into a clear, bulleted list for the user.
    - Museum: {museum_result}
    - Concert: {concert_result}
    - Restaurant: {restaurant_result}
    """
)

# ✨ The SequentialAgent runs the parallel search, then the synthesis ✨
parallel_planner_agent = SequentialAgent(
    name="parallel_planner_agent",
    sub_agents=[parallel_research_agent, synthesis_agent],
    description="A workflow that finds multiple things in parallel and then summarizes the results."
)

print("🤖 Agent team supercharged with a ParallelAgent workflow!")

🤖 Agent team supercharged with a ParallelAgent workflow!


---
### Final Step: Updating the Router and Running the App

Now we just have one last thing to do: make our `router_agent` aware of these powerful new workflows! We'll add `iterative_planner_agent` and `parallel_planner_agent` to its list of available options.

Then we can run our app with new queries designed to trigger these advanced, multi-agent workflows.

```
                    +---------------------+
                    |    User Query 🗣️     |
                    +----------+----------+
                               |
                               v
                    +---------------------+
                    |   Router Agent 🤖    |
                    |  (Classify Request) |
                    +----------+----------+
                               |
      +-----------+-----------+-----------+-----------+------------+
      |           |           |           |           |            |
      v           v           v           v           v            v
+-------------+  +------------------+  +------------------+  +------------------+  +-----------------+
| foodie_agent|  | find_and_navigate|  | iterative_planner|  | parallel_planner |  | day_trip_agent  |
| 🍣 Food Only |  | 🧭 Seq Workflow   |  | 🔁 Loop Workflow  |  | ⚡ Parallel Tasks |  | 🧳 Basic Plan     |
+-------------+  +------------------+  +------------------+  +------------------+  +-----------------+
```

In [18]:
# --- The ULTIMATE Router Agent --- #

router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a master request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about finding a single food place.
    - 'find_and_navigate_agent': For queries that ask to *first find a place* and *then get directions* to it.
    - 'iterative_planner_agent': For planning a trip with a specific constraint that needs checking, like travel time.
    - 'parallel_planner_agent': For queries that ask to find multiple, independent things at once (e.g., a museum AND a concert AND a restaurant).
    - 'day_trip_agent': A general planner for any other simple day trip requests.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# The master dictionary of all our executable agents and workflows
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent, # For simple food queries
    "find_and_navigate_agent": find_and_navigate_agent, # Sequential
    "iterative_planner_agent": iterative_planner_agent, # Loop
    "parallel_planner_agent": parallel_planner_agent,   # Parallel
}

# --- Let's Test Everything! ---

async def run_fully_loaded_app():
    queries = [
        # Test Case 1: Simple Sequential Flow
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.",

        # Test Case 2: Iterative Loop Flow
        "Plan me a day in San Francisco with a museum and a nice dinner, but make sure the travel time between them is very short.",

        # Test Case 3: Parallel Flow
        "Help me plan a trip to SF. I need one museum, one concert, and one great restaurant."
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        if chosen_route in worker_agents:
            worker_agent = worker_agents[chosen_route]
            print(f"--- Handing off to {worker_agent.name} ---")
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
            print(f"--- {worker_agent.name} Complete ---")
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_fully_loaded_app()


🗣️ Processing New Query: 'Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'e8baf3f4-8451-490b-97db-0b5d5607cc3a'...
🚦 Router has selected route: 'An error occurred: 
On how to mitigate this issue, please refer to:

https://google.github.io/adk-docs/agents/models/#error-code-429-resource_exhausted


429 RESOURCE_EXHAUSTED. {error: {code: 429, message: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 42.898501562s., status: RESOURCE_EXHAUSTED, details: [{@type: type.googleapis.com/google.

---
## 🎉 Congratulations! 🎉

You've completed the Enhanced ADK Adventure! You have successfully. Let's review the advanced orchestration patterns you've successfully implemented:

- **The Router Pattern**: You built a master router agent capable of analyzing user intent and delegating tasks to the appropriate specialist agent or workflow.

- **Sequential Workflows**: Using SequentialAgent, you elegantly chained agents together, creating clean, readable code for multi-step tasks without manual data handling.

- **Iterative Refinement**: You constructed a sophisticated feedback loop with LoopAgent, enabling your agents to plan, self-critique, and improve their output until it met specific constraints.

- **Parallel Power**: You maximized speed and efficiency by using ParallelAgent to run multiple research tasks concurrently, later synthesizing the results into a unified response.


```
   __            /\_/\         /\_/\        /\_/\         __             (\__/)
o-''|\_____/).  ( o.o )       ( -.- )      ( ^_^ )     o-''|\_____/).    ( ^_^ )
 \_/|_)     )    > ^ <         > * <        >💖<         \_/|_)     )     / >🌸< \
    \  __  /                                              \  __  /         /   \
    (_/ (_/                                               (_/ (_/        (___|___)
```
